In [1]:
from numpy import array
from keras.callbacks import CSVLogger
import numpy as np
from pickle import dump
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import GRU
from keras.layers import Embedding
import io

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
def load_doc(filename):
    with io.open(filename, 'r', encoding='utf8') as f:
        text = f.read()
    return text

In [4]:
file_path = "/content/drive/My Drive/next-word-prediction/data/data2.txt"
doc = load_doc(file_path)
lines = doc.split('\n')

In [5]:
lines

['a guide to',
 'guide to word',
 'to word embedding',
 'word embedding with',
 'embedding with gensim',
 'with gensim model',
 'gensim model handson',
 'model handson graph',
 'handson graph neural',
 'graph neural networks',
 'neural networks with',
 'networks with pytorch',
 'with pytorch pytorch',
 'pytorch pytorch geometric',
 'pytorch geometric how',
 'geometric how to',
 'how to use',
 'to use in',
 'use in python',
 'in python databricks',
 'python databricks how',
 'databricks how to',
 'how to save',
 'to save files',
 'save files in',
 'files in csv',
 'in csv on',
 'csv on your',
 'on your local',
 'your local computer',
 'local computer a',
 'computer a stepbystep',
 'a stepbystep implementation',
 'stepbystep implementation of',
 'implementation of gradient',
 'of gradient descent',
 'gradient descent and',
 'descent and backpropagation',
 'and backpropagation an',
 'backpropagation an easy',
 'an easy introduction',
 'easy introduction to',
 'introduction to sql',
 'to s

In [6]:
# integer encode sequences of words
tokenizer = Tokenizer()
tokenizer.fit_on_texts(lines)
sequences = tokenizer.texts_to_sequences(lines)
# vocabulary size
vocab_size = len(tokenizer.word_index) + 1

In [7]:
vocab_size

7761

In [8]:
sequences

[[3, 58, 2],
 [58, 2, 400],
 [2, 400, 1409],
 [400, 1409, 11],
 [1409, 11, 2310],
 [11, 2310, 86],
 [2310, 86, 3472],
 [86, 3472, 674],
 [3472, 674, 69],
 [674, 69, 90],
 [69, 90, 11],
 [90, 11, 301],
 [11, 301, 301],
 [301, 301, 1731],
 [301, 1731, 5],
 [1731, 5, 2],
 [5, 2, 56],
 [2, 56, 7],
 [56, 7, 35],
 [7, 35, 3473],
 [35, 3473, 5],
 [3473, 5, 2],
 [5, 2, 232],
 [2, 232, 1410],
 [232, 1410, 7],
 [1410, 7, 2311],
 [7, 2311, 17],
 [2311, 17, 8],
 [17, 8, 855],
 [8, 855, 233],
 [855, 233, 3],
 [233, 3, 856],
 [3, 856, 434],
 [856, 434, 4],
 [434, 4, 1732],
 [4, 1732, 1173],
 [1732, 1173, 6],
 [1173, 6, 1411],
 [6, 1411, 20],
 [1411, 20, 174],
 [20, 174, 105],
 [174, 105, 2],
 [105, 2, 401],
 [2, 401, 9],
 [401, 9, 13],
 [9, 13, 302],
 [13, 302, 1412],
 [302, 1412, 266],
 [1412, 266, 3474],
 [266, 3474, 105],
 [3474, 105, 2],
 [105, 2, 3475],
 [2, 3475, 675],
 [3475, 675, 2312],
 [675, 2312, 676],
 [2312, 676, 267],
 [676, 267, 343],
 [267, 343, 1733],
 [343, 1733, 12],
 [1733, 12, 1

In [9]:
length = 3

In [10]:
print(len(sequences))
print(sequences[0][0])

48547
3


In [11]:
sequences = array(sequences)
sequences

array([[   3,   58,    2],
       [  58,    2,  400],
       [   2,  400, 1409],
       ...,
       [  55,    3,   74],
       [   3,   74,   47],
       [  74,   47,  673]])

In [12]:
sequences.size

145641

In [13]:
c=1
h=0
while(c!=0):
    limit = len(sequences)
    d =[]
    c=0
    for i in range(0 , limit):
        l=len(sequences[i])
        if(l!=length):
            print(sequences[i])
            sequences = np.delete(sequences, i, 0)
            
            c=1
            print("error at ", i)
            print("Deleting :_________________________ ") 
            h = h+1
            break        
print("Total deletion: ", h)

Total deletion:  0


In [14]:
# separate into input and output
sequences = array(sequences)
X, y = sequences[:,:-1], sequences[:,-1]
y = to_categorical(y, num_classes=vocab_size)
seq_length = X.shape[1]

In [15]:
# define model
# model = Sequential()
# model.add(Embedding(vocab_size, 50, input_length=seq_length))
# model.add(GRU(100, return_sequences=True))
# model.add(GRU(100))
# model.add(Dense(100, activation='relu'))
# model.add(Dense(vocab_size, activation='softmax'))
# print(model.summary())
# # compile model
# model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# # fit model
# csv_logger = CSVLogger('training_1_1.log')
# model.fit(X, y, batch_size=500, epochs=1000, callbacks=[csv_logger])

In [17]:
# model.save('/content/drive/My Drive/next-word-prediction/models/model_2.h5')
# dump(tokenizer, open(r'/content/drive/My Drive/next-word-prediction/models/tokenizer_2.pkl', 'wb'))

In [18]:
from pickle import load
from keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [31]:
# generate a sequence from a language model
def generate_seq(model, tokenizer, seq_length, seed_text, n_words):
    result = list()
    in_text = seed_text
    # generate a fixed number of words
    for _ in range(n_words):
        # encode the text as integer
        encoded = tokenizer.texts_to_sequences([in_text])[0]
        # truncate sequences to a fixed length
        encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre')
        # predict probabilities for each word
        yhat = model.predict(encoded, verbose=0)
        # map predicted word index to word
        out_word = ''
        for word, index in tokenizer.word_index.items():
            if index == yhat.any():
                out_word = word
                break
        # append to input
        in_text += ' ' + out_word
        result.append(out_word)
    return ' '.join(result)

In [32]:
model_1 = load_model(r'/content/drive/My Drive/next-word-prediction/models/model_1.h5')
tokenizer_1 = load(open(r'/content/drive/My Drive/next-word-prediction/models/tokenizer_1.pkl', 'rb'))

In [33]:
model_2 = load_model(r'/content/drive/My Drive/next-word-prediction/models/model_2.h5')
tokenizer_2 = load(open(r'/content/drive/My Drive/next-word-prediction/models/tokenizer_2.pkl', 'rb'))

In [34]:
#seed_text = lines[randint(0,len(lines))]
#print(seed_text + '\n')
seed_text = 'data'
res = len(seed_text.split())
if(res>5):
    res=3
    seed_text = seed_text.split()
    lent = len(seed_text)
    lent = lent-3
    seed_text = seed_text[lent:]
    seed_text = ' '.join(map(str, seed_text))


seq_length = res



if(res==1):
    model=model_1
    tokenizer = tokenizer_1
elif(res==2):
    #For 2nd prediction
    model_new = model_1
    tokenizer_new = tokenizer_1
    #For 1st prediction
    model=model_2
    tokenizer = tokenizer_2
     
print('Inputed words: ' + '\n'+ seed_text + '\n')
#Predict Next Word(1)
print('Predicted next word: ')
generated_word_1 = generate_seq(model, tokenizer, seq_length, seed_text, 1)
print(generated_word_1, end =" ")

Inputed words: 
data

Predicted next word: 
the 